In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
import matplotlib.ticker as mtick
%matplotlib qt

In [2]:
path = '/media/onno/Algemeen/Thesis/GFS_T850/GSS/'

In [19]:

lead_dayz = [1,3,5,7,9]
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw']
seasonz = ['DJF','MAM','JJA','SON']
titlez = ['Persistent Heatwaves','Short-Lived Heatwaves',
         ' Persistent Coldwaves','Short-Lived Coldwaves']
xtickz = np.arange(4)
file = 'GSS_Temp_Extremes_Seasonally_Grouped_lead_day_{}_{}.csv'
for lead_day in lead_dayz:
    df = pd.read_csv(path+file.format(lead_day,fcst_model),index_col=0)
    fig,axz = plt.subplots(2,2,figsize=(16,9))
    fig.suptitle('Gilbert Skill Score Day {} Forecast Before Onset of Temperature Extreme'.format(lead_day))
    for i,ax in enumerate(axz.flat):
        df_sub=df[columnz[i]]
        ax.set_title(titlez[i])
        ax.set_ylim([0,1])
        ax.bar(0,df_sub.loc['DJF'],label='DJF',color='dodgerblue')
        ax.bar(1,df_sub.loc['MAM'],label='MAM',color='limegreen')
        ax.bar(2,df_sub.loc['JJA'],label='DJF',color='r')
        ax.bar(3,df_sub.loc['SON'],label='SON',color='darkorange')
        ax.grid(axis='y')
        ax.set_xticks(xtickz)
        ax.set_xticklabels(seasonz)
        for k in range(len(df_sub.values)):
            ax.text(xtickz[k],0.08,'{:.2f}'.format(df_sub.loc[seasonz[k]]),fontsize=18,color='w',horizontalalignment='center')
            
        ax.legend()
    sys.exit()
    fig.savefig(path+'GSS_seasonally_grouped_lead_day_{}.png'.format(lead_day))
    plt.close(fig)
sys.exit()
        

SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
lat_0 = 46
lat_1 = 52
lon_0 = 6
lon_1 = 12
fcst_model = 'ERA5RF'
model_dic = {'ERA5RF':'ERA5 Reforecast',
            'GFS':'GEFS Reforecast'}
lead_dayz = [1,3,5,7,9]
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw']

titlez = ['Persistent Heatwaves','Short-Lived Heatwaves',
         ' Persistent Coldwaves','Short-Lived Coldwaves']
xtickz = np.arange(4)
lead_dayz = [1,3,5,7,9]
file = 'Gilbert_Skill_Score_lead_day_{}.csv'
seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}

for lead_day in lead_dayz:
    df = pd.read_csv(path+file.format(lead_day),index_col=0)
    df.index = pd.to_datetime(df.index)
#     df_seasonally = df.groupby(month_to_season_lu[df.index.month]).mean()
    fig,axz = plt.subplots(2,2,figsize=(16,9))
    fig.suptitle('Gilbert Skill Score Day {} Forecast Before Onset of Temperature Extreme {} {}-{} °N {}-{} °E'.format(lead_day,
                                                                                                    model_dic[fcst_model],
                                                                                                    lat_0,lat_1+2,lon_0,lon_1+2))
    fig.subplots_adjust(left=0.05,right=0.875,bottom=0.05,top=0.9)
    for i,ax in enumerate(axz.flat):
        df_sub=df[columnz[i]].dropna()
        ax.set_title(titlez[i])
        ax.set_ylim([0,1])
        
        dataz = [df_sub[np.isin(df_sub.index.month,seasonz['DJF'])],
               df_sub[np.isin(df_sub.index.month,seasonz['MAM'])],
                df_sub[np.isin(df_sub.index.month,seasonz['JJA'])],
                df_sub[np.isin(df_sub.index.month,seasonz['SON'])]]

        for x in xtickz:
            GSS_mean = dataz[x].mean()
            GSS_worst = ((dataz[x]<0.2)).sum()/dataz[x].count()
            GSS_bad = ((dataz[x]>=0.2)&(dataz[x]<0.4)).sum()/dataz[x].count()
            GSS_meh = ((dataz[x]>=0.4)&(dataz[x]<0.6)).sum()/dataz[x].count()
            GSS_good = ((dataz[x]>=0.6)&(dataz[x]<0.8)).sum()/dataz[x].count()
            GSS_perfect = ((dataz[x]>=0.8)&(dataz[x]<=1)).sum()/dataz[x].count()
            if x==0:
                ax.bar(x,GSS_worst,width=0.5,color='darkred',label='GSS < 0.2')
                ax.bar(x,GSS_bad,width=0.5,color='red',label='0.4 > GSS > 0.2',
                       bottom=GSS_worst)
                ax.bar(x,GSS_meh,width=0.5,color='orange',label='0.6 > GSS > 0.4',
                      bottom=GSS_worst+GSS_bad)
                ax.bar(x,GSS_good,width=0.5,color='yellow',label='0.8 > GSS > 0.6',
                      bottom=GSS_worst+GSS_bad+GSS_meh)
                ax.bar(x,GSS_perfect,width=0.5,color='lime',label='1 > GSS > 0.8',
                      bottom=GSS_worst+GSS_bad+GSS_meh+GSS_good)
            else:
                ax.bar(x,GSS_worst,width=0.5,color='darkred')
                ax.bar(x,GSS_bad,width=0.5,color='red',
                       bottom=GSS_worst)
                ax.bar(x,GSS_meh,width=0.5,color='orange',
                      bottom=GSS_worst+GSS_bad)
                ax.bar(x,GSS_good,width=0.5,color='yellow',
                      bottom=GSS_worst+GSS_bad+GSS_meh)
                ax.bar(x,GSS_perfect,width=0.5,color='lime',
                      bottom=GSS_worst+GSS_bad+GSS_meh+GSS_good)
            ax.text(xtickz[x],0.05,'{:.2f}'.format(GSS_mean),
                    fontsize=18,color='w',horizontalalignment='center')
            ax.text(xtickz[x],0.92,'{}'.format(len(dataz[x])),
                    fontsize=18,color='k',horizontalalignment='center')
#         ax.bar(0,df_sub.loc['DJF'],label='DJF',color='dodgerblue')
#         ax.bar(1,df_sub.loc['MAM'],label='MAM',color='limegreen')
#         ax.bar(2,df_sub.loc['JJA'],label='DJF',color='r')
#         ax.bar(3,df_sub.loc['SON'],label='SON',color='darkorange')
        ax.grid(axis='y')
        ax.set_xticks(xtickz)
        ax.set_xticklabels(['DJF','MAM','JJA','SON'])
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
        ax.set_ylabel('% of Forecasts')

        if i ==1:
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.8275))
    fig.savefig(path+'GSS_seasonally_grouped_lead_day_{}_{}.png'.format(lead_day,model_dic[fcst_model]),dpi=150)
    plt.close(fig)
        